In [11]:
import pandas as pd
import psycopg2
import configparser
import plotly.express as px
import plotly.graph_objects as go
import re

In [12]:
# Load config
config = configparser.ConfigParser()
config.read('Config.txt')

# Get database configurations
db_config = config['DATABASE']

In [13]:
import psycopg2

try:
    cnx = psycopg2.connect(
        user=db_config['USER'],
        password=db_config['PASSWORD'],
        host=db_config['HOST'],
        port=db_config['PORT'],
        database=db_config['NAME']
    )

    print("Connected successfully!")
    cur = cnx.cursor()

    # Query to fetch all table names in the current schema
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")

    # Fetch all table names
    table_names = cur.fetchall()

    # Print the table names
    for table in table_names:
        print(table[0])

    cnx.close()

except Exception as e:
    print(f"Error: {e}")


Connected successfully!
music_charts
music_charts_productie
merged_data
music_charts_global_productie
incidentdata
music_charts_streams


In [18]:
import psycopg2

try:
    cnx = psycopg2.connect(
        user=db_config['USER'],
        password=db_config['PASSWORD'],
        host=db_config['HOST'],
        port=db_config['PORT'],
        database=db_config['NAME']
    )

    print("Connected successfully!")
    cur = cnx.cursor()

    # Specify the table name
    table_name = 'music_charts_productie'

    # Query to fetch the first 20 rows from the specified table
    query = f"SELECT * FROM {table_name} LIMIT 20;"

    # Execute the query
    cur.execute(query)

    # Fetch all rows
    rows = cur.fetchall()

    # Print the column names
    cur.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}';")
    column_names = [column[0] for column in cur.fetchall()]
    print(column_names)

    # Print the first 20 rows
    for row in rows:
        print(row)

    cnx.close()

except Exception as e:
    print(f"Error: {e}")


Connected successfully!
['id', 'valence', 'artist', 'track_id', 'language', 'week', 'song']
(None, 'СОЛНЦЕ МОНАКО (Tha Remix)', 'Lusia Chebotina, blago white, MAYOT', '2021-11-26', 0.748, '56vbR6vkOTNRehnuv8gNBF', 'Russian')
(None, 'Гуси', 'Wellboy', '2021-11-26', 0.663, '4TLYCLkGimyHZDaLFMLKOc', 'Ukrainian')
(None, 'Rampampam', 'Minelli', '2021-11-26', 0.437, '1esZgaqm0YKN87CIP3orJ3', 'English')
(None, 'Заебавшие рожи', 'Grigory Leps', '2021-11-26', 0.266, '7dLs0hSjodwgs1htpFFSUC', 'Russian')
(None, 'Міраж', 'Artem Pivovarov', '2021-11-26', 0.704, '3FOBoSvg3paiwcGfOUQLqj', 'Ukrainian')
(None, 'Вишні - New Year Version', 'Wellboy', '2021-11-26', 0.607, '0pomtUMMZwdYLog9ZdqrJ7', 'Ukrainian')
(None, 'Федерико Феллини', 'Galibri & Mavik', '2021-11-26', 0.936, '7Eyfmn0zcgcjGUrWlFdgqa', 'Russian')
(None, 'Americano', 'LOBODA', '2021-11-26', 0.256, '3VGOgdTqoMYOohpmD0IIHL', 'Russian')
(None, 'Million', 'Big Baby Tape, kizaru', '2021-11-26', 0.719, '5Z8YDahqRCzMQRwKZPk5dC', 'Russian')
(None, 

In [14]:
def connect_to_db():
    try:
        cnx = psycopg2.connect(
            user=db_config['USER'],
            password=db_config['PASSWORD'],
            host=db_config['HOST'],
            port=db_config['PORT'],
            database=db_config['NAME']
        )
        print("Connected successfully!")
        return cnx
    except Exception as e:
        print(f"Error: {e}")
        return None

In [20]:
def create_monthly_valence_chart_figure():
    music_df = fetch_music_data()
    if music_df is not None:
        # Convert 'week' column to datetime format if it's not already
        music_df['week'] = pd.to_datetime(music_df['week'])
        
        # Extracting the month and year from the 'week' column
        music_df['month_year'] = music_df['week'].dt.to_period('M').astype(str)

        # Grouping by month and calculating the average valence
        monthly_avg_valence = music_df.groupby('month_year')['average_valence'].mean().reset_index()

        fig = px.line(monthly_avg_valence, x='month_year', y='average_valence', title='Monthly Average Valence Over Time')

        # Define valence ranges for different moods and corresponding mood labels
        valence_ranges = [(0, 0.3), (0.3, 0.6), (0.6, 1.0)]  # You can adjust these ranges
        mood_labels = ['Sad', 'Neutral', 'Happy']  # Mood labels corresponding to valence ranges

        # Define colors for different moods
        mood_colors = ['red', 'blue', 'green']

        # Assign colors based on valence ranges and specify mood labels in the legend
        for i, (start, end) in enumerate(valence_ranges):
            fig.add_shape(type='rect', x0=min(monthly_avg_valence['month_year']), x1=max(monthly_avg_valence['month_year']),
                          y0=start, y1=end,
                          fillcolor=mood_colors[i], opacity=0.2, layer='below', line_width=0, name=mood_labels[i])

        fig.update_traces(marker=dict(size=10, opacity=0.6), selector=dict(mode='markers'))
        fig.update_xaxes(title_text='Month')
        fig.update_yaxes(title_text='Average Monthly Valence')
        fig.update_layout(showlegend=True)  # Show the legend

        return fig


Connected successfully!
